### Train a machine learning model 

#### Step 1: Setup 

In [1]:
#%pip install pandas numpy -qU

In [2]:
#%pip install scikit-learn -qU

In [3]:
# %pip install shap -qU

In [4]:
 #%pip install xgboost -qU

In [5]:
# %pip install  matplotlib -qU

In [6]:
 #%pip install numpy -qU

In [7]:
# %pip install catboost -qU

In [8]:
# %pip install lightgbm -qU

In [9]:
# %pip install seaborn -qU

In [10]:
# %pip install optuna -qU

In [11]:
# !nvidia-smi

In [12]:
import pandas as pd
import os
import sklearn
import shap
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.model_selection import train_test_split, cross_validate,RandomizedSearchCV, StratifiedKFold
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance
import matplotlib.pyplot as plt
from trainMLmodel import surfaceDesignTopAndBottom,selectDescriptors,combineRankAndDescriptorsClassyfication,explainer, combineRankAndDescriptorsClassificationbotisFLAT, combineRankAndDescriptorsClassyfication_Danilingus
import numpy as np

In [13]:
cd

/bme001.mnt/home/20212358


#### Step 2: load the data file

In [14]:
# Define the location of the csv files imageOutliersRemoved and Ranking
#pathToImageObject=os.getcwd()+"/DataAnalysis/"
pathToImageObject="/bme001.mnt/home/20212358"
locationOfImageObject1="cleaned_rawImagedataFI_final.csv"
locationOfImageObject2="DataAnalysis_withJan/Screen_Mean_Mean_FilteredNuclei_new_AreaShape_FormFactor.csv"
# Define the location of the annotation files
pathToTopoUnitImages="/FeatureImages/FeatureImages/"
allTopoDescriptors=pd.read_csv("allTopoDescriptors.csv")
# check if file exists
if os.path.isfile(locationOfImageObject1):
    imagedataOutliersRemoved=pd.read_csv(locationOfImageObject1,low_memory=False)
    if os.path.isfile(locationOfImageObject2):
        imageDataRankedSurfaces=pd.read_csv(locationOfImageObject2,low_memory=False)
    else:
        print("File not found, check if you finished ranking the features")
else:
    print('File not find, check if you removed the outliers')


In [15]:
#imagedataOutliersRemoved["Mean_Nephrin_Granularity_8_CorrCellBody"]

In [16]:
#replacing ranking by previous module with ranking sorted from lowest hit to the highest
#locationOfImageObject2=pathToImageObject+"/DataAnalysis/Screen_Mean_Mean_Nephrin_AreaShape_MajorAxisLength.csv"
# locationOfImageObject2=pathToImageObject+"/DataAnalysis/PostBlindTestHits/Screen_Mean_Mean_Nephrin_Granularity_8_CorrCellBody.csv"
# imageDataRankedSurfaces=pd.read_csv(locationOfImageObject2,low_memory=False)

In [17]:
imageDataRankedSurfaces

FeatureIdx  \
0            209   
1            336   
2           1413   
3            836   
4            299   
...          ...   
1787        1846   
1788        1484   
1789         116   
1790        1100   
1791        1336   

      Screen_ADpvalue_Mean_FilteredNuclei_new_AreaShape_FormFactor  \
0                                              0.205240              
1                                              0.250000              
2                                              0.149180              
3                                              0.158281              
4                                              0.250000              
...                                                 ...              
1787                                           0.250000              
1788                                           0.177034              
1789                                           0.250000              
1790                                           0.250000              
1791                                           0.228384              

      Screen_KSpvalue_Mean_FilteredNuclei_new_AreaShape_FormFactor  \
0                                              0.575175              
1                                              0.372805              
2                                              0.660140              
3                                              0.141414              
4                                              0.962704              
...                                                 ...              
1787                                           0.660140              
1788                                           0.242452              
1789                                           0.980109              
1790                                           0.660140              
1791                                           0.252214              

      Screen_SNR_Mean_FilteredNuclei_new_AreaShape_FormFactor  \
0                                             17.210070         
1                                             16.417523         
2                                             19.257066         
3                                             19.336009         
4                                             14.124505         
...                                                 ...         
1787                                          12.484857         
1788                                           6.565678         
1789                                           8.394622         
1790                                          14.665901         
1791                                           7.692446         

      Screen_Mean_Mean_FilteredNuclei_new_AreaShape_FormFactor  \
0                                              0.741277          
1                                              0.737256          
2                                              0.736993          
3                                              0.736736          
4                                              0.736028          
...                                                 ...          
1787                                           0.604139          
1788                                           0.603946          
1789                                           0.597855          
1790                                           0.597663          
1791                                           0.594988          

      Screen_SD_Mean_FilteredNuclei_new_AreaShape_FormFactor  \
0                                              0.043072        
1                                              0.044907        
2                                              0.038271        
3                                              0.038102        
4                                              0.052110        
...                                                 ...        
1787                                           0.048390      

In [18]:
# locationOfImageObject3=pathToImageObject+"/DataAnalysis/Screen_Mean_Count_CellBody_new_Ranking.csv"
# imageDataRankedCount=pd.read_csv(locationOfImageObject3,low_memory=False)
# imageDataRankedCount

#### Step 3: Select Features for the machine learning.

##### Step 3.1 Vizualize the topographies design from top and bottom of the rank.

First, select the feature of interest

In [19]:
featureOfInterest='Screen_Mean_Mean_FilteredNuclei_new_AreaShape_FormFactor'

Second, select the number of top and bottom ranked feautures you want to select

In [20]:
numberOfRankedFeaturesToVisualize=100

In [21]:
#surfaceDesignTopAndBottom(imageDataRankedSurfaces,pathToTopoUnitImages,featureOfInterest,number=numberOfRankedFeaturesToVisualize)
imagedataOutliersRemoved

AreaOccupied_AreaOccupied_CellBody_new  \
0                                    195441.0   
1                                    190993.0   
2                                    157466.0   
3                                    128276.0   
4                                    159374.0   
...                                       ...   
31915                                146459.0   
31916                                113561.0   
31917                                136466.0   
31918                                108352.0   
31919                                132796.0   

       AreaOccupied_AreaOccupied_FilteredNuclei_new  \
0                                           41149.0   
1                                           34146.0   
2                                           21593.0   
3                                           30686.0   
4                                           36365.0   
...                                             ...   
31915                                       28944.0   
31916                                       21965.0   
31917                                       22229.0   
31918                                       18080.0   
31919                                       35925.0   

       AreaOccupied_AreaOccupied_FilteredNuclei_target  \
0                                              38157.0   
1                                              33484.0   
2                                              19752.0   
3                                              28584.0   
4                                              35031.0   
...                                                ...   
31915                                          24587.0   
31916                                          19472.0   
31917                                          18386.0   
31918                                          12223.0   
31919                                          27669.0   

       AreaOccupied_AreaOccupied_MaskedNuclei  \
0                                    147590.0   
1                                    178264.0   
2                                    112684.0   
3                                    115514.0   
4                                    147877.0   
...                                       ...   
31915                                128723.0   
31916                                 96719.0   
31917                                109961.0   
31918                                 73657.0   
31919                                106458.0   

       AreaOccupied_AreaOccupied_Nephrin  AreaOccupied_Perimeter_CellBody_new  \
0                               166158.0                              17129.0   
1                               210194.0                              18661.0   
2                               133496.0                              13267.0   
3                               138323.0                              11218.0   
4                               200466.0                              16275.0   
...                                  ...                                  ...   
31915                           165853.0                              13291.0   
31916                           153661.0                              10378.0   
31917                           143216.0                              12602.0   
31918                           146310.0                              10716.0   
31919                           133671.0                              10817.0   

       AreaOccupied_Perimeter_FilteredNuclei_new  \
0                                         6242.0   
1                                         5837.0   
2                                         3797.0   
3                                         4632.0   
4                                         5380.0   
...                                          ...   
31915                                     4030.0   
31916                                     3448.0   
31917                         

### Step 4 Load and vizualize descriptors data

##### Step 4.1 select descriptors based on a keyword 

In [22]:
selectedDescriptors= selectDescriptors(allDescriptors=allTopoDescriptors,
             # keywordsToInclude=["Kamiel"],
                 # keywordsToExclude=["_Index","Height","Width", "Center","Texture","ImageNumber","Zernike","sd","Count","Number","var","AngleBetweenNeighbors","mad","Neighbors"])
#selectedDescriptors= selectDescriptors(allDescriptors=allTopoDescriptors,
              #  keywordsToInclude=["Pattern","Kamiel"],
                 #keywordsToExclude=["_Index","Height","Width", "Center","Number","Zernike"])
   keywordsToExclude=["_Index","Height","Width", "Center","Number","Zernike","Texture"])

In [23]:
pd.options.display.max_columns = 30
selectedDescriptors

FeatureIdx  Pattern_AreaOccupied_AreaOccupied_Pattern  \
0              1                                       5215   
1              2                                       2604   
2              3                                      20366   
3              4                                       4864   
4              5                                      17246   
...          ...                                        ...   
2171        2172                                       3821   
2172        2173                                      16244   
2173        2174                                       1855   
2174        2175                                       4794   
2175        2176                                      23548   

      Pattern_AreaOccupied_Perimeter_Pattern  \
0                                        330   
1                                        422   
2                                        910   
3                                        439   
4                                       1252   
...                                      ...   
2171                                     436   
2172                                    1497   
2173                                     306   
2174                                     492   
2175                                    1632   

      Pattern_AreaOccupied_TotalArea_Pattern  Pattern_Count_Pattern  \
0                                      40000                      1   
1                                      10000                      1   
2                                      40000                      1   
3                                      10000                      1   
4                                      40000                      1   
...                                      ...                    ...   
2171                                   10000                      1   
2172                                   78400                      2   
2173                                   10000                      1   
2174                                   10000                      2   
2175                                   78400                      1   

      Pattern_AreaShape_Area_sd  Pattern_AreaShape_Compactness_sd  \
0                         0.000                          0.000000   
1                         0.000                          0.000000   
2                         0.000                          0.000000   
3                         0.000                          0.000000   
4                         0.000                          0.000000   
...                         ...                               ...   
2171                      0.000                          0.000000   
2172                   3183.395                          0.654076   
2173                      0.000                          0.000000   
2174                   1681.500                          0.306309   
2175                      0.000                          0.000000   

      Pattern_AreaShape_Eccentricity_sd  Pattern_AreaShape_Extent_sd  \
0                              0.000000                     0.000000   
1                              0.000000                     0.000000   
2                              0.000000                     0.000000   
3                              0.000000                     0.000000   
4                              0.000000                     0.000000   
...                                 ...                          ...   
2171                           0.000000                     0.000000   
2172                           0.052437                     0.022692   
2173                           0.000000                     0.000000   
2174                           0.553171                     0.158168   
2175                           0.000000                     0.000000   

      Pattern_AreaShape_FormFactor_sd  Pattern_AreaShape_MajorAxisLength_sd  \
0                            0.000000         

In [24]:
Screen_Mean_Mean_Nephrin_Granularity_1 = pd.read_csv("Screen_Mean_Mean_Nephrin_Granularity_1_CorrCellBody.csv")
Screen_Mean_Mean_Nephrin_Granularity_1

FeatureIdx  Screen_ADpvalue_Mean_Nephrin_Granularity_1_CorrCellBody  \
0           1658                                           0.250000         
1            189                                           0.083959         
2           1270                                           0.095665         
3            122                                           0.250000         
4           1353                                           0.250000         
...          ...                                                ...         
2086        1836                                           0.163023         
2087        1860                                           0.250000         
2088        1394                                           0.250000         
2089         757                                           0.069970         
2090        1292                                           0.250000         

      Screen_KSpvalue_Mean_Nephrin_Granularity_1_CorrCellBody  \
0                                              0.730111         
1                                              0.126748         
2                                              0.055944         
3                                              0.647552         
4                                              0.700123         
...                                                 ...         
2086                                           0.078568         
2087                                           0.860428         
2088                                           0.730111         
2089                                           0.078568         
2090                                           0.893706         

      Screen_SNR_Mean_Nephrin_Granularity_1_CorrCellBody  \
0                                              5.508653    
1                                              5.717872    
2                                              4.253026    
3                                              5.190216    
4                                              4.866214    
...                                                 ...    
2086                                           5.657096    
2087                                           5.332652    
2088                                           5.167565    
2089                                           6.501527    
2090                                           3.432248    

      Screen_Mean_Mean_Nephrin_Granularity_1_CorrCellBody  \
0                                             46.026999     
1                                             45.273450     
2                                             45.259933     
3                                             45.068389     
4                                             44.444154     
...                                                 ...     
2086                                          30.269535     
2087                                          29.868709     
2088                                          29.775861     
2089                                          29.053096     
2090                                          28.751350     

      Screen_SD_Mean_Nephrin_Granularity_1_CorrCellBody  \
0                                              8.355400   
1                                              7.917885   
2                                             10.641819   
3                                              8.683335   
4                                              9.133211   
...                                                 ...   
2086                                           5.350720   
2087                                           5.601099   
2088                                           5.762068   
2089                                           4.468657   
2090                                           8.376829   

      Screen_Median_Mean_Nephrin_Granularity_1_CorrCellBody    Rank  
0                                             45.283829         1.0

### Step 5: Train Machine Learning models

##### Step 5.1 : train the model based on the ranked surfaces and the descriptors selected in step 4.1

In [25]:
imageDataRankedSurfaces

FeatureIdx  \
0            209   
1            336   
2           1413   
3            836   
4            299   
...          ...   
1787        1846   
1788        1484   
1789         116   
1790        1100   
1791        1336   

      Screen_ADpvalue_Mean_FilteredNuclei_new_AreaShape_FormFactor  \
0                                              0.205240              
1                                              0.250000              
2                                              0.149180              
3                                              0.158281              
4                                              0.250000              
...                                                 ...              
1787                                           0.250000              
1788                                           0.177034              
1789                                           0.250000              
1790                                           0.250000              
1791                                           0.228384              

      Screen_KSpvalue_Mean_FilteredNuclei_new_AreaShape_FormFactor  \
0                                              0.575175              
1                                              0.372805              
2                                              0.660140              
3                                              0.141414              
4                                              0.962704              
...                                                 ...              
1787                                           0.660140              
1788                                           0.242452              
1789                                           0.980109              
1790                                           0.660140              
1791                                           0.252214              

      Screen_SNR_Mean_FilteredNuclei_new_AreaShape_FormFactor  \
0                                             17.210070         
1                                             16.417523         
2                                             19.257066         
3                                             19.336009         
4                                             14.124505         
...                                                 ...         
1787                                          12.484857         
1788                                           6.565678         
1789                                           8.394622         
1790                                          14.665901         
1791                                           7.692446         

      Screen_Mean_Mean_FilteredNuclei_new_AreaShape_FormFactor  \
0                                              0.741277          
1                                              0.737256          
2                                              0.736993          
3                                              0.736736          
4                                              0.736028          
...                                                 ...          
1787                                           0.604139          
1788                                           0.603946          
1789                                           0.597855          
1790                                           0.597663          
1791                                           0.594988          

      Screen_SD_Mean_FilteredNuclei_new_AreaShape_FormFactor  \
0                                              0.043072        
1                                              0.044907        
2                                              0.038271        
3                                              0.038102        
4                                              0.052110        
...                                                 ...        
1787                                           0.048390      

In [26]:
imageDataRankedSurfaces.shape

(1792, 8)

In [27]:
# #pd.merge(left=imageDataRankedSurfaces, right=result_df,
#                       how='left', left_on="FeatureIdx", 
#                       right_on="FeatureIdx").Rank_x

In [28]:
X,Y=combineRankAndDescriptorsClassyfication(SortedRank=imageDataRankedSurfaces,Descriptors=selectedDescriptors,numberPerClass=89)

X.shape

(178, 383)

In [29]:
Y=Y.astype(bool)

In [30]:
def reduce_memory_usage(df):
    import gc
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()
                
                if (df[col].nunique()==2):
                    df[col] = df[col].astype(bool)

                elif str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
            else:
                if (df[col].nunique()==2):
                    df[col] = df[col].astype(bool)
                    
                if(df[col].nunique()>100):
                    df[col] = df[col].astype('category')
                else:
                    if((col_type != 'datetime64[ns]')):
                        df[col] = df[col].astype(str)
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage became: ",mem_usg," MB")
    gc.collect()
    return df

X=reduce_memory_usage(X)

Memory usage of dataframe is 0.52 MB
Memory usage became:  0.13405799865722656  MB


##### Step 5.2: split X and Y in training and test data set

In [31]:
a=0
b=0
for i in Y.values:
    if (i==False):
        b+=1
    else:
        a+=1
a,b

(89, 89)

In [32]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,stratify=Y )
# X_train,X_val,y_train,y_val=train_test_split(X_train,y_train)

In [33]:
X_test,X_val,y_test,y_val=train_test_split(X_test,y_test,test_size=0.2,stratify=y_test )


##### Step 5.3: select a classification model

In [34]:
import lightgbm as lgb
import joblib
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score


def objective(trial,X_train =X_train ,y_train = y_train, X_test = X_test,y_test=y_test, big_df = X, big_y=Y ):

    param = {
    'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart','rf']), #, 'rf' 
    'num_leaves': trial.suggest_int('num_leaves', 2, 2500),
    'max_depth':trial.suggest_int('max_depth', -10, 900),
    'objective':trial.suggest_categorical('objective', ['binary']),
    'learning_rate': trial.suggest_float('learning_rate', 1e-12, 0.99, log=False),
    'n_estimators': trial.suggest_int('n_estimators', 100, 24_100),
    #'subsample_for_bin': trial.suggest_int('subsample_for_bin', 50_000, 450_000, 50_000),
    'random_state': trial.suggest_categorical('random_state', [42]),
    'data_sample_strategy':trial.suggest_categorical('data_sample_strategy', ['bagging','goss']),
    # 'tree_learner':trial.suggest_categorical('tree_learner', ['serial', 'feature', 'data', 'voting']),
    'deterministic':trial.suggest_categorical('deterministic', [True]),
    'force_col_wise':trial.suggest_categorical('force_col_wise', [True,False]),
    # 'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 2_230),
    
    'min_sum_hessian_in_leaf': trial.suggest_float('min_sum_hessian_in_leaf',  0, 100),
# #     'bagging_fraction': trial.suggest_float('bagging_fraction',  1e-12, 1),
    'feature_fraction': trial.suggest_float('feature_fraction', 1e-12, 0.9, log=False),
    'feature_fraction_bynode': trial.suggest_float('feature_fraction_bynode', 1e-12, 0.9, log=False),
    'extra_trees':trial.suggest_categorical('extra_trees', [True,False]),
    'lambda_l1': trial.suggest_float('lambda_l1',  0, 100),
    'lambda_l2': trial.suggest_float('lambda_l2',  0, 100),
    'linear_lambda': trial.suggest_float('linear_lambda',  0, 100),
    'drop_rate': trial.suggest_float('drop_rate',  0, 1),
    'max_drop': trial.suggest_int('max_drop', 0, 2000),
    'skip_drop': trial.suggest_float('skip_drop',  0, 1),
    'uniform_drop':trial.suggest_categorical('uniform_drop', [True,False]),
    'top_rate': trial.suggest_float('top_rate',  0, 0.5),
    'other_rate': trial.suggest_float('other_rate',  0, 0.5),
    'min_data_per_group': trial.suggest_int('min_data_per_group', 1, 2000),
    'max_cat_threshold': trial.suggest_int('max_cat_threshold', 1, 2000),
    'cat_l2': trial.suggest_float('cat_l2',  0, 100),
    'cat_smooth': trial.suggest_float('cat_smooth',  0, 100),
    'top_k': trial.suggest_int('top_k', 1, 2000),
    'cegb_tradeoff': trial.suggest_float('cegb_tradeoff',  0, 100),
    # 'cegb_penalty_split': trial.suggest_float('cegb_penalty_split',  0, 100),
    # 'path_smooth': trial.suggest_float('path_smooth',  0, 100),
    # 'num_grad_quant_bins': trial.suggest_int('num_grad_quant_bins', 1, 2000),
#     'is_unbalance':trial.suggest_categorical('uniform_drop', [True,False]),
    'scale_pos_weight': trial.suggest_float('scale_pos_weight',  1e-12, 100),
    'sigmoid': trial.suggest_float('sigmoid',  1e-12, 100),
    # 'boost_from_average':trial.suggest_categorical('boost_from_average', [True,False]),
    'max_bin':trial.suggest_int('max_bin', 50, 1_000),
        
    'n_jobs':trial.suggest_categorical('n_jobs', [-1]), 
    'verbose': trial.suggest_categorical('verbose', [-1]), 
    }


    model = lgb.LGBMClassifier(**param)
    model.fit(X_train, y_train, eval_set=[
              (X_test,y_test)])


    y_pred_test = model.predict(X_test)
    # print(y_pred_test)
    

    #accuracy_test = balanced_accuracy_score(y_pred_test, y_test)

    #accuracy_test = (sklearn.model_selection.cross_val_score (model, big_df, big_y, cv=5, scoring='balanced_accuracy', n_jobs=1)).mean()
    # try:
    #     accuracy_test = roc_auc_score(y_pred_test, y_test)
    # except Exception:
    #     accuracy_test=0

    try:
        predicted_train = model.predict(X_train)
        probabilities_train = model.predict_proba(X_train)
        predicted_test = model.predict(X_test)
        probabilities_test = model.predict_proba(X_test)
            
        train_fpr, train_tpr, train_threshold = metrics.roc_curve(y_train, probabilities_train[:,1])
        test_fpr, test_tpr, test_threshold = metrics.roc_curve(y_test, probabilities_test[:,1])
    
        accuracy_train = metrics.auc(train_fpr, train_tpr)
        accuracy_test = metrics.auc(test_fpr, test_tpr)

        bal_accuracy_train = balanced_accuracy_score(predicted_train, y_train)
        bal_accuracy_test = balanced_accuracy_score(predicted_test, y_test) 
    except Exception:
        accuracy_test=0
        accuracy_train = 0
        bal_accuracy_test = 0
        bal_accuracy_train=0


    joblib.dump(study, 'N_FormFactor_another_lgbm_with_val.pkl')

    if (int(trial.number)%50==0):
        print("BACKUP GOES BRRRRR")
        joblib.dump(study, 'N_FormFactor_another_lgbm_with_val_BACKUP.pkl')


    return accuracy_test, accuracy_train,bal_accuracy_train,bal_accuracy_test

In [35]:
# https://vaex.io/docs/tutorial_ml.html#CatBoost-example

In [36]:
def logging_callback(study, frozen_trial):
    # Retrieve previous best values for both objectives
    previous_best_value_0 = study.user_attrs.get("previous_best_value_0", None)
    previous_best_value_1 = study.user_attrs.get("previous_best_value_1", None)
    previous_best_value_2 = study.user_attrs.get("previous_best_value_2", None)
    previous_best_value_3 = study.user_attrs.get("previous_best_value_3", None)

    
    # Get the current trial's values
    current_values = frozen_trial.values

    if current_values is None:
        return  # Skip if the trial doesn't have valid results

    # current_value_0, current_value_1 = current_values
    current_value_0, current_value_1,current_value_2,current_value_3 = current_values


    # Check if both objectives improved
    if (
        (previous_best_value_0 is None or current_value_0 >= previous_best_value_0) and
        (previous_best_value_1 is None or current_value_1 >= previous_best_value_1) and
        (previous_best_value_2 is None or current_value_2 >= previous_best_value_2) and
        (previous_best_value_3 is None or current_value_3 >= previous_best_value_3) 
    ):
        # Update the best values in the study's user attributes
        study.set_user_attr("previous_best_value_0", current_value_0)
        study.set_user_attr("previous_best_value_1", current_value_1)
        study.set_user_attr("previous_best_value_2", current_value_2)
        study.set_user_attr("previous_best_value_3", current_value_3)

        # Log the improved metrics
        print(40*"*",)
        print("\n\n\n")
        print(
            f"\nNew best trial with both objectives improved!\n"
            f"Trial {frozen_trial.number}:\n"
            f"  Objective 0: {current_value_0}\n"
            f"  Objective 1: {current_value_1}\n"
            f"  Objective 2: {current_value_2}\n"
            f"  Objective 3: {current_value_3}\n"
            f"  Params: {frozen_trial.params}\n"
        )
        print("\n\n\n")

        print(40*"*","\n")


In [37]:
#!rm N_FormFactor_another_lgbm_with_val.pkl

In [38]:
!free -th

              total        used        free      shared  buff/cache   available
Mem:          503Gi       9.2Gi       406Gi       4.9Gi        87Gi       485Gi
Swap:         3.7Gi       3.7Gi       1.0Mi
Total:        507Gi        12Gi       406Gi


In [39]:
#plotting SHAP

import shap
import matplotlib.pyplot as plt
import joblib
import lightgbm as lgb

plt.close('all')
# Load the Optuna study
study = joblib.load('N_FormFactor_another_lgbm_with_val.pkl')  # Ensure the correct path

# Retrieve the best trial from the Pareto front
best_trial = study.best_trials[0]  # Select the first Pareto-optimal trial
best_params = best_trial.params

print("Best Parameters:", best_params)

# Load the dataset (ensure these variables are defined in your notebook)
X_train = X_train  # Replace with actual training data variable
y_train = y_train  # Replace with actual training labels variable
X_test = X_test    # Replace with actual testing data variable

# Train the LightGBM model with the best parameters
model = lgb.LGBMClassifier(**best_params)  # Use LGBMRegressor if regression
model.fit(X_train, y_train)

# Use TreeExplainer for LightGBM models
explainer = shap.TreeExplainer(model)

# Compute SHAP values
shap_values = explainer.shap_values(X_test)

# Plot and save the SHAP summary plot
shap.summary_plot(shap_values, X_test, show=False)
plt.savefig("shap_summary_plot.png", bbox_inches='tight')
print("SHAP summary plot saved as 'shap_summary_plot.png'")


Best Parameters: {'boosting_type': 'gbdt', 'num_leaves': 2344, 'max_depth': 143, 'objective': 'binary', 'learning_rate': 0.160839184175993, 'n_estimators': 19096, 'random_state': 42, 'data_sample_strategy': 'bagging', 'deterministic': True, 'force_col_wise': True, 'min_sum_hessian_in_leaf': 24.510336946654864, 'feature_fraction': 0.800147235827867, 'feature_fraction_bynode': 0.13100195956003666, 'extra_trees': True, 'lambda_l1': 81.11613026633972, 'lambda_l2': 63.050402187691866, 'linear_lambda': 91.92050676719862, 'drop_rate': 0.5015053380297326, 'max_drop': 661, 'skip_drop': 0.7721270831877045, 'uniform_drop': False, 'top_rate': 0.11268141057481107, 'other_rate': 0.27760236901042024, 'min_data_per_group': 896, 'max_cat_threshold': 1258, 'cat_l2': 92.67257503587795, 'cat_smooth': 85.79697472853832, 'top_k': 615, 'cegb_tradeoff': 55.539820505123096, 'scale_pos_weight': 1.60063435484666, 'sigmoid': 71.71476934052164, 'max_bin': 113, 'n_jobs': -1, 'verbose': -1}


/home/20212358/.local/lib/python3.10/site-packages/shap/explainers/_tree.py:448: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn('LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray')


SHAP summary plot saved as 'shap_summary_plot.png'


In [40]:
import joblib

# Load Optuna study
study = joblib.load('N_FormFactor_another_lgbm_with_val.pkl')
plt.close('all')
# Get the best trial parameters
best_trial = study.best_trials[0]
best_params = best_trial.params

# Train the model with the best parameters
from lightgbm import LGBMClassifier
bestClassyficationModel = LGBMClassifier(**best_params)
bestClassyficationModel.fit(X_train, y_train)


from lightgbm import plot_importance
import matplotlib.pyplot as plt

# Plot feature importance
fig, ax = plt.subplots(figsize=(10, 10))
plot_importance(bestClassyficationModel, max_num_features=15, ax=ax, importance_type='split')
plt.title("Top 15 Features by Importance")

plt.show()
plt.savefig("Top 15 Features by Importance", bbox_inches='tight')

In [85]:
%%time 


#optuna.logging.set_verbosity(optuna.logging.WARNING)

import optuna, gc
gc.collect()

if os.path.isfile('N_FormFactor_another_lgbm_with_val.pkl'):
    print("Found file !")
    study = joblib.load('N_FormFactor_another_lgbm_with_val.pkl')
else:
    print("CREATING NEW FILE...")
    sampler = optuna.samplers.TPESampler(multivariate=True, warn_independent_sampling=False)
    study = optuna.create_study(directions=['maximize', 'maximize','maximize','maximize'], load_if_exists=True,sampler=sampler,pruner=optuna.pruners.MedianPruner(n_warmup_steps=30,n_startup_trials=20))
study.optimize(objective, timeout=1_000_000, gc_after_trial=True,
              show_progress_bar=True, n_jobs=1, callbacks=[logging_callback])
print("Finished working")
joblib.dump(study, 'N_FormFactor_another_lgbm_with_val.pkl')

Found file !


   0%|          | 00:00/277:46:40

[I 2025-04-02 17:22:45,934] Trial 52 finished with values: [0.5612244897959184, 0.8807776234874033, 0.7945759967617891, 0.675] and parameters: {'boosting_type': 'rf', 'num_leaves': 1179, 'max_depth': 268, 'objective': 'binary', 'learning_rate': 0.7943437899385867, 'n_estimators': 10446, 'random_state': 42, 'data_sample_strategy': 'goss', 'deterministic': True, 'force_col_wise': False, 'min_sum_hessian_in_leaf': 46.63847008607457, 'feature_fraction': 0.6332300399561965, 'feature_fraction_bynode': 0.5669594755711855, 'extra_trees': False, 'lambda_l1': 0.6461026047937035, 'lambda_l2': 65.58289531551006, 'linear_lambda': 81.16206205143668, 'drop_rate': 0.21747635028960316, 'max_drop': 1697, 'skip_drop': 0.7918292215098782, 'uniform_drop': False, 'top_rate': 0.44739985516687775, 'other_rate': 0.1922117878216117, 'min_data_per_group': 1357, 'max_cat_threshold': 869, 'cat_l2': 81.99646518656269, 'cat_smooth': 2.315088666969423, 'top_k': 73, 'cegb_tradeoff': 2.5067273253673594, 'scale_pos_weig

/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[I 2025-04-02 17:24:59,810] Trial 60 finished with values: [0.5025510204081632, 0.6635588176949019, 0.5, 0.5] and parameters: {'boosting_type': 'rf', 'num_leaves': 494, 'max_depth': 88, 'objective': 'binary', 'learning_rate': 0.4992518535590797, 'n_estimators': 19618, 'random_state': 42, 'data_sample_strategy': 'bagging', 'deterministic': True, 'force_col_wise': True, 'min_sum_hessian_in_leaf': 29.371748181874857, 'feature_fraction': 0.5382920226013974, 'feature_fraction_bynode': 0.3907968704351585, 'extra_trees': False, 'lambda_l1': 39.90406588058353, 'lambda_l2': 78.47989710792864, 'linear_lambda': 57.924728231476394, 'drop_rate': 0.20350898493253194, 'max_drop': 1159, 'skip_drop': 0.7759725485631099, 'uniform_drop': False, 'top_rate': 0.4400340285081031, 'other_rate': 0.2736557545221249, 'min_data_per_group': 1750, 'max_cat_threshold': 1291, 'cat_l2': 11.479611407288708, 'cat_smooth': 70.12908356902969, 'top_k': 1807, 'cegb_tradeoff': 95.30883783860705, 'scale_pos_weight': 32.142703

...

/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[I 2025-04-02 18:05:30,668] Trial 380 finished with values: [0.5459183673469388, 0.8694703431858759, 0.5, 0.5] and parameters: {'boosting_type': 'rf', 'num_leaves': 2364, 'max_depth': 282, 'objective': 'binary', 'learning_rate': 0.7692530796311481, 'n_estimators': 19468, 'random_state': 42, 'data_sample_strategy': 'goss', 'deterministic': True, 'force_col_wise': False, 'min_sum_hessian_in_leaf': 25.199327388863935, 'feature_fraction': 0.6889758061222857, 'feature_fraction_bynode': 0.13130468599825185, 'extra_trees': False, 'lambda_l1': 4.988982679997292, 'lambda_l2': 97.07615599722021, 'linear_lambda': 39.58935749505132, 'drop_rate': 0.7561652993024299, 'max_drop': 1684, 'skip_drop': 0.9220351623540997, 'uniform_drop': False, 'top_rate': 0.13258351744769925, 'other_rate': 0.08768791911290608, 'min_data_per_group': 823, 'max_cat_threshold': 30, 'cat_l2': 69.07232578733995, 'cat_smooth': 73.58795217258317, 'top_k': 122, 'cegb_tradeoff': 84.3135864899717, 'scale_pos_weight': 1.68152199594

/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[I 2025-04-02 18:05:31,838] Trial 381 finished with values: [0.5, 0.5, 0.5, 0.5] and parameters: {'boosting_type': 'gbdt', 'num_leaves': 2072, 'max_depth': 324, 'objective': 'binary', 'learning_rate': 0.36652931102386344, 'n_estimators': 6174, 'random_state': 42, 'data_sample_strategy': 'goss', 'deterministic': True, 'force_col_wise': False, 'min_sum_hessian_in_leaf': 45.89032190896631, 'feature_fraction': 0.008361293781077472, 'feature_fraction_bynode': 0.4749570043287193, 'extra_trees': True, 'lambda_l1': 4.493445736087168, 'lambda_l2': 36.57821492104697, 'linear_lambda': 48.05287407442216, 'drop_rate': 0.2654246893932583, 'max_drop': 306, 'skip_drop': 0.09397392708255636, 'uniform_drop': False, 'top_rate': 0.26347779731121623, 'other_rate': 0.1102698344617653, 'min_data_per_group': 1290, 'max_cat_threshold': 28, 'cat_l2': 9.477942611047027, 'cat_smooth': 80.96054306618201, 'top_k': 1420, 'cegb_tradeoff': 7.85585954056954, 'scale_pos_weight': 87.21826814048987, 'sigmoid': 3.291278439

KeyboardInterrupt: 

In [ ]:
print(1/0)
#CHECK TRIAL 1580, 1709, 1845, 4669, 9338

#after second december 2024
#check 643 (0.17,0.94);  765 (0.16, 1.0);  801 (0.154, 1.0);

In [86]:
study = joblib.load('N_FormFactor_another_lgbm_with_val.pkl')



In [87]:
# trial_number = 4669
# specific_trial = next((t for t in study.trials if t.number == trial_number), None)

# if specific_trial is not None:
#     print(f"Trial #{specific_trial.number}")
#     print("Parameters:", specific_trial.params)
#     print("Value:", specific_trial.values)
# else:
#     print(f"Trial #{trial_number} not found.")

In [88]:
print('Number of finished trials:', len(study.trials))
try:
    print('Best trial:', study.best_trial.params)
except Exception:
    pass

Number of finished trials: 386


In [89]:
try:
    Best_trial = study.best_trial.params
    #Best_trial['task_type']='GPU'
    Best_trial['verbose'] = -1
    print(Best_trial)
except Exception:
    Best_trial = study.best_trials[0].params
    #Best_trial['task_type']='GPU'
    Best_trial['verbose'] = -1
    print(Best_trial)

{'boosting_type': 'gbdt', 'num_leaves': 1112, 'max_depth': 40, 'objective': 'binary', 'learning_rate': 0.7419249627177356, 'n_estimators': 22888, 'random_state': 42, 'data_sample_strategy': 'bagging', 'deterministic': True, 'force_col_wise': True, 'min_sum_hessian_in_leaf': 39.59421319684793, 'feature_fraction': 0.7009662315625628, 'feature_fraction_bynode': 0.7421699701875102, 'extra_trees': False, 'lambda_l1': 14.393409333748039, 'lambda_l2': 34.002755826973086, 'linear_lambda': 97.9628525592144, 'drop_rate': 0.5076264079004207, 'max_drop': 1289, 'skip_drop': 0.9915949755488966, 'uniform_drop': True, 'top_rate': 0.2930815119593869, 'other_rate': 0.39593680647586277, 'min_data_per_group': 1582, 'max_cat_threshold': 1927, 'cat_l2': 18.368206838145998, 'cat_smooth': 59.640617667709535, 'top_k': 1462, 'cegb_tradeoff': 46.261797849161624, 'scale_pos_weight': 25.975224662660246, 'sigmoid': 65.50703554774955, 'max_bin': 122, 'n_jobs': -1, 'verbose': -1}


In [90]:
for i in range(len(study.best_trials)):
    print(study.best_trials[i].values)

[0.7448979591836735, 1.0, 1.0, 0.8181818181818181]


In [45]:
try:
    print(study.best_trial.value)
except Exception:
    print(study.best_trials[0].values)

[0.6523809523809524, 1.0, 1.0, 0.7283653846153846]


###### study.best_trials[1].params

In [91]:
%%capture

"""
try:
    best_model = lgb.LGBMClassifier(**study.best_params)
    best_model.fit(X_train, y_train, eval_set=[
                  (X_test,y_test)])
except Exception:
    print("Using alternative")
    best_model = lgb.LGBMClassifier(**study.best_trials[0].params)
    best_model.fit(X_train, y_train, eval_set=[
                  (X_test,y_test)])

"""

best_model = lgb.LGBMClassifier(**study.best_trials[0].params)
#best_model = lgb.LGBMClassifier(**specific_trial.params)
best_model.fit(X_train, y_train, eval_set=[
              (X_test,y_test)])

In [92]:
# best_model = cb.CatBoostClassifier(**study.best_params,
#                                 ).fit(X, Y, verbose=False, early_stopping_rounds=500)

In [93]:
best_model.predict(X_test)

array([ True,  True,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True, False,  True, False,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
       False])

In [94]:
predicted_train = best_model.predict(X_train)
probabilities_train = best_model.predict_proba(X_train)
predicted_test = best_model.predict(X_test)
probabilities_test = best_model.predict_proba(X_test)

#FOR VALIDATION
predicted_val = best_model.predict(X_val)
probabilities_val = best_model.predict_proba(X_val)

In [95]:
y_test.nunique()

2

In [96]:
from sklearn.metrics import balanced_accuracy_score

display(balanced_accuracy_score(y_test, predicted_test))
display(balanced_accuracy_score(y_train, predicted_train))

0.7142857142857143

1.0

In [97]:
# FOR VALIDATION

display(balanced_accuracy_score(y_val, predicted_val))

0.75

In [98]:
# calculate the false and true positive rates (fpr, tpr)
train_fpr, train_tpr, train_threshold = metrics.roc_curve(y_train, probabilities_train[:,1])
test_fpr, test_tpr, test_threshold = metrics.roc_curve(y_test, probabilities_test[:,1])
#obtain the values for the AUC plot
train_roc_auc = metrics.auc(train_fpr, train_tpr)
test_roc_auc = metrics.auc(test_fpr, test_tpr)
print(train_roc_auc, test_roc_auc)
#make the plot
fig, ax = plt.subplots(figsize=[5,5])
plt.title('Receiver Operating Characteristic')
plt.plot(train_fpr, train_tpr, 'b', label = 'Train AUC = %0.2f' % train_roc_auc)
plt.plot(test_fpr, test_tpr, 'g', label = 'Test AUC = %0.2f' % test_roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.savefig('N_FormFactor_another_lgbm_with_val.png')
plt.show()


1.0 0.7448979591836735


In [133]:
#-------------------

In [134]:
#together with Koen
bestClassyficationModel=RandomForestClassifier(**bestParameters)
bestClassyficationModel.fit(X_train, y_train)

NameError: name 'RandomForestClassifier' is not defined

In [135]:
predicted_train = bestClassyficationModel.predict(X_train)
probabilities_train = bestClassyficationModel.predict_proba(X_train)
predicted_test = bestClassyficationModel.predict(X_test)
probabilities_test = bestClassyficationModel.predict_proba(X_test)

NameError: name 'bestClassyficationModel' is not defined

Step 5.7 :Print out accuracy metrics

In [ ]:
print ('\nTrain Accuracy:', metrics.accuracy_score(y_train, predicted_train))
print ('Test Accuracy:', metrics.accuracy_score(y_test, predicted_test))


Step 5.8: plot the AUC curve for training and test data

In [ ]:
# calculate the false and true positive rates (fpr, tpr)
train_fpr, train_tpr, train_threshold = metrics.roc_curve(y_train, probabilities_train[:,1])
test_fpr, test_tpr, test_threshold = metrics.roc_curve(y_test, probabilities_test[:,1])
#obtain the values for the AUC plot
train_roc_auc = metrics.auc(train_fpr, train_tpr)
test_roc_auc = metrics.auc(test_fpr, test_tpr)
#make the plot
fig, ax = plt.subplots(figsize=[5,5])
plt.title('Receiver Operating Characteristic')
plt.plot(train_fpr, train_tpr, 'b', label = 'Train AUC = %0.2f' % train_roc_auc)
plt.plot(test_fpr, test_tpr, 'g', label = 'Test AUC = %0.2f' % test_roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Step 5.9: Plot 10 most important features for the model

In [ ]:
plot_importance(bestClassyficationModel,max_num_features=20,xlabel='Features weight');    
plt.show()

### Step 6: Use SHAP module to explain predicted model

##### Step 6.1: Show which features contribute to prediction of particular surface

In [ ]:
explainerDefinition=explainer(dataForModel=X,
          model=bestClassyficationModel,
          hit=1)

##### Step 6.2: Show SHAP values for every important feature and every predicted surface Plot explanation https://shap.readthedocs.io/en/latest/

In [ ]:
shap_values = explainerDefinition.shap_values(X)
shap.summary_plot(shap_values, X)

##### Step 6.3: Draw a scatter plot with features of interest

Set the feature of interest for the x- and y-axis

In [ ]:
xFeature="Kamiel_FCP"
yFeature='InsCirclRadius_median'

In [ ]:
fig, ax = plt.subplots(figsize=[10,10])

colors = {0:'orange', 1:'blue'}

binned=X
binned["Class"]=Y

grouped = binned.groupby("Class")

for key, group in grouped:
    group.plot(ax=ax, kind='scatter', 
               x=xFeature, 
               y=yFeature, 
               label=key, color=colors[key])

plt.show()

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import numpy as np

# 1. Handle Class Imbalance with SMOTE
smote = SMOTE(random_state=1)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# 2. Scale Features
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

# 3. Hyperparameter Search Space Refinement
searchParameters = {
    'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'max_depth': [5, 10, 20],
    'learning_rate': [0.0001, 0.001, 0.01],
    'n_estimators': [100, 250, 500],
    'reg_alpha': [0.01, 0.1, 1],
    'reg_lambda': [0.01, 0.1, 1]
}

# 4. Updated Fitting Parameters
fittingParameters = {
    "early_stopping_rounds": 50,
    "eval_metric": "auc",
    "eval_set": [(X_test_scaled, y_test)]  # Using validation set for early stopping
}

# 5. Classification Model
classificationModel = XGBClassifier(objective='binary:logistic', random_state=1)

# 6. Randomized Search with Cross-Validation
random_search = RandomizedSearchCV(
    estimator=classificationModel,
    param_distributions=searchParameters,
    n_iter=50,  # Reduce iterations for quicker testing; increase later
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=10,
    random_state=1
)

# 7. Fit the Model
searchModel = random_search.fit(X_resampled_scaled, y_resampled, **fittingParameters)

# 8. Best Parameters
bestParameters = searchModel.best_estimator_.get_params()
print("Best Parameters:", bestParameters)

# 9. Evaluate Model on Test Data
y_pred = searchModel.predict(X_test_scaled)
test_auc = roc_auc_score(y_test, y_pred)
train_auc = roc_auc_score(y_resampled, searchModel.predict(X_resampled_scaled))

print(f"Train AUC: {train_auc}")
print(f"Test AUC: {test_auc}")
